In [7]:
! git clone https://github.com/facebookresearch/fastText.git
! cd fastText
! pip install fastText
# import fasttext.util

fatal: destination path 'fastText' already exists and is not an empty directory.


  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Running setup.py clean for fastText
Failed to build fastText
    Running setup.py install for fastText: started
    Running setup.py install for fastText: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\HP\AppData\Local\Programs\Python\Python310\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\HP\\AppData\\Local\\Temp\\pip-install-tg418gyr\\fasttext_a33082c31331487488af0d0c3c64f671\\setup.py'"'"'; __file__='"'"'C:\\Users\\HP\\AppData\\Local\\Temp\\pip-install-tg418gyr\\fasttext_a33082c31331487488af0d0c3c64f671\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\HP\AppData\Local\Temp\pip-wheel-tqzpyt0p'
       cwd: C:\Users\HP\AppData\Local\Temp\pip-install-tg418gyr\fasttext_a33082c31331487488af0d0c3c64f671\
  Complete output (20 lines):
  C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\dist.py:697: 

In [8]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
import fasttext.utils

# Download the FastText model
fasttext.util.download_model('en', if_exists='ignore')

# Load the FastText model
ft = fasttext.load_model('cc.en.300.bin')

# Function to load FastText embeddings
def load_fasttext_embeddings(model, word_to_index, embedding_dim):
    embedding_matrix = np.zeros((len(word_to_index) + 1, embedding_dim))

    for word, index in word_to_index.items():
        if word in model:
            embedding_matrix[index] = model[word]
        else:
            # Use a default value (e.g., zero vector) when the word is not in the model
            embedding_matrix[index] = np.zeros(embedding_dim)

    return embedding_matrix

# Load processed data
with open("processed_train_data.json", "r") as f_train, open("processed_val_data.json", "r") as f_val:
    processed_train_data = json.load(f_train)
    processed_val_data = json.load(f_val)

texts_train = [entry["text"] for entry in processed_train_data.values()]
labels_train = [entry["labels"] for entry in processed_train_data.values()]

texts_val = [entry["text"] for entry in processed_val_data.values()]
labels_val = [entry["labels"] for entry in processed_val_data.values()]

# Build word_to_index
word_to_index = {}
index = 1
for entry in processed_train_data.values():
    tokens = entry["text"].split()
    for token in tokens:
        if token not in word_to_index:
            word_to_index[token] = index
            index += 1

word_to_index["<unk>"] = 0
embedding_dim_fasttext = 100  
embedding_matrix_fasttext = load_fasttext_embeddings(ft, word_to_index, embedding_dim_fasttext)

label_to_index = {'O': 0, 'B': 1, 'I': 2}

X_train = [[word_to_index.get(token, word_to_index["<unk>"]) for token in text.split()] for text in texts_train]
y_train = [[label_to_index[label] for label in entry] for entry in labels_train]
X_val = [[word_to_index.get(token, word_to_index["<unk>"]) for token in text.split()] for text in texts_val]
y_val = [[label_to_index[label] for label in entry] for entry in labels_val]

# Dataset and DataLoader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.LongTensor(self.X[idx]), torch.LongTensor(self.y[idx])

def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs, targets

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Model definition
class SimpleRNNFastText(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size):
        super(SimpleRNNFastText, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), padding_idx=0)
        self.rnn = nn.RNN(input_size=embedding_dim_fasttext, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.rnn(x)
        output = self.fc(output)
        return output
# Initialize model, criterion, and optimizer
hidden_size = 100
output_size = len(label_to_index)
model_fasttext = SimpleRNNFastText(embedding_matrix_fasttext, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fasttext.parameters(), lr=0.001)

# Training loop
num_epochs = 60
train_losses = []
val_losses = []
train_f1_scores = []
val_f1_scores = []

for epoch in range(num_epochs):
    model_fasttext.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model_fasttext(inputs)
        loss = criterion(outputs.view(-1, output_size), targets.view(-1))
        loss.backward()
        optimizer.step()

    model_fasttext.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        val_loss = 0.0
        for inputs, targets in val_loader:
            outputs = model_fasttext(inputs)
            loss = criterion(outputs.view(-1, output_size), targets.view(-1))
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=2).cpu().numpy()
            targets = targets.cpu().numpy()

            all_preds.extend(preds)
            all_targets.extend(targets)

        val_loss /= len(val_loader)
        val_macro_f1 = f1_score(np.concatenate(all_targets, axis=0), np.concatenate(all_preds, axis=0), average='macro')

        train_losses.append(loss.item())
        val_losses.append(val_loss)
        train_f1_scores.append(val_macro_f1)
        val_f1_scores.append(val_macro_f1)

        if epoch == num_epochs - 1:
            print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}')
            print(f'Validation Macro F1: {val_macro_f1:.4f}')

# Plotting Loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.savefig('plots/rnn_fasttext/Loss.png')  # Save the plot
plt.show()

# Plotting F1 Score
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_f1_scores, label='Training Macro-F1')
plt.plot(range(1, num_epochs+1), val_f1_scores, label='Validation Macro-F1')
plt.xlabel('Epochs')
plt.ylabel('Macro-F1 Score')
plt.title('Training and Validation Macro-F1 Score Over Epochs')
plt.legend()
plt.savefig('plots/rnn_fasttext/Macro.png')  # Save the plot
plt.show()

# Save the model
torch.save(model_fasttext.state_dict(), "models/rnn_fasttext.pth")

ModuleNotFoundError: No module named 'fasttext'